### ✅ **Wrapper**

> A **Wrapper** is like a helper that **talks to a website** (like Arxiv or Wikipedia) and **gets the data**.

🧠 Think: "Go and get the info!"

---

### ✅ **QueryRun**

> A **QueryRun** is a tool that **asks a question** and uses the wrapper to **get the answer**.

🧠 Think: "Ask and get the answer back!"

---

### 🎯 Real-Life Example:

* **Wrapper** = goes to Arxiv and brings research papers.
* **QueryRun** = asks: “What are the latest ML papers?” and shows the results using the wrapper.

---

### 📝 Summary:

| Name         | What it does                             |
| ------------ | ---------------------------------------- |
| **Wrapper**  | Gets data from a website                 |
| **QueryRun** | Uses the wrapper to answer your question |


In [1]:
##Arxiv----- research papers
#Tools creation 
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper,WikipediaAPIWrapper

In [ ]:
# Creating wikipedia_wrapper
wikipedia_wrapper= WikipediaAPIWrapper(
    top_k_results=1,
    doc_content_chars_max=250
)

#Creating wikipedia tool using wikipedia_wrapper which is inbuilt
wikipedia_tool= WikipediaQueryRun(api_wrapper=wikipedia_wrapper)
wikipedia_tool.name

'wikipedia'

In [4]:
# Similarly creating arxiv_wrapper and arxiv_tool

arxiv_wrapper= ArxivAPIWrapper(
    top_k_results=1,
    doc_content_chars_max=250
)

arxiv_tool= ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv_tool.name

'arxiv'

# Combining tools

In [ ]:
tools= [wikipedia_tool,arxiv_tool]

In [7]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [8]:
loader= WebBaseLoader("https://docs.smith.langchain.com")
docs= loader.load()
text_splitter= RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

texts= text_splitter.split_documents(docs)

vectorstore= FAISS.from_documents(texts, OpenAIEmbeddings())

retriever= vectorstore.as_retriever()

retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001915449B880>, search_kwargs={})

In [10]:
# creating own retriever tool using the retriever 
# Convert reteriever to a tool,so that it can used as a tool
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(
    retriever=retriever,
    name="langsmith-search",
    description="Useful for retrieving information from LangChain documentation.",
)

retriever_tool.name

'langsmith-search'

In [12]:
tools= [wikipedia_tool,arxiv_tool,retriever_tool]
print(tools[0].name, tools[1].name, tools[2].name)
tools


wikipedia arxiv langsmith-search


[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\GENERATIVE AI\\Langchain projects\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith-search', description='Useful for retrieving information from LangChain documentation.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001917EBA95A0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS 

In [13]:
## Run all the tools with Agents and LLM Models

## Tools,LLM---->AgentExecutor
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()
import openai
import os 

groqApiKey= os.getenv("GROQ_API_KEY")
openai.api_key= os.getenv("OPENAI_API_KEY")

llm=ChatGroq(groq_api_key=groqApiKey, model="Llama3-8b-8192")
llm 

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000019156B04730>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000019156C4C100>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [14]:
## Prompt template using hub 
from langchain import hub 
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [15]:
# Agents 
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm=llm,tools=tools,prompt=prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [16]:
# For running the agent, we can use AgentExecutor
from langchain.agents import AgentExecutor 
agent_executor= AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)

In [17]:
agent_executor.invoke({"input": "What is LangChain?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'LangChain'}`


Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of languag
Invoking: `langsmith-search` with `{'query': 'langchain integration framework'}`


Skip to main contentWe are growing and hiring for multiple roles for LangChain, LangGraph and LangSmith. Join our team!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceChangelogCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions F

{'input': 'What is LangChain?',
 'output': 'The information provided about LangChain has been helpful in understanding what LangChain is. LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications.'}

In [18]:
agent_executor.invoke({"input": "What is Machine learning?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks witIt seems that the tool yielded a result, and I can continue the conversation without using any more tools.

Machine learning is a subfield of artificial intelligence that involves the use of algorithms and statistical models to enable machines to learn from data and make predictions or decisions. The goal of machine learning is to develop systems that can automatically improve their performance on a task over time, without being explicitly programmed. This is achieved by training the system on a dataset, which allows it to learn patterns and relationships in the data.

Some common applications of machine learning include image and speech re

{'input': 'What is Machine learning?',
 'output': 'It seems that the tool yielded a result, and I can continue the conversation without using any more tools.\n\nMachine learning is a subfield of artificial intelligence that involves the use of algorithms and statistical models to enable machines to learn from data and make predictions or decisions. The goal of machine learning is to develop systems that can automatically improve their performance on a task over time, without being explicitly programmed. This is achieved by training the system on a dataset, which allows it to learn patterns and relationships in the data.\n\nSome common applications of machine learning include image and speech recognition, natural language processing, recommender systems, and predictive modeling. Machine learning has many potential uses, including healthcare, finance, and education, and is a rapidly growing field with many new developments and applications being explored.'}

In [23]:
agent_executor.invoke({"input": "tell me about the paper arXiv:1706.03762 ?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on c
Invoking: `wikipedia` with `{'query': 'Attention Is All You Need'}`


Page: Attention Is All You Need
Summary: "Attention Is All You Need" is a 2017 landmark research paper in machine learning authored by eight scientists working at Google. The paper introduced a new deep learning architecture known as the transformer,The paper "Attention Is All You Need" by Vaswani et al. introduced the transformer architecture, which has since become a widely used and influential model in natural language processing and other areas of machine learning.

> Finished chain.


{'input': 'tell me about the paper arXiv:1706.03762 ?',
 'output': 'The paper "Attention Is All You Need" by Vaswani et al. introduced the transformer architecture, which has since become a widely used and influential model in natural language processing and other areas of machine learning.'}

 **definitions**:

# Tool in LangChain:

A **Tool** is a function that does a specific task, like doing math, searching the web, or running code. It helps the chatbot get things done.

# Agent in LangChain:

An **Agent** is a smart controller that decides which tool to use and in what order to answer a user's question.


Here's a **real-world scenario** with a clear example of a **Tool** and an **Agent**:


##  **Scenario: Planning a Trip**

You ask a chatbot:

> "Book a flight from New York to Los Angeles, check the weather there, and suggest what to pack."


###  **Tools Involved**:

1.  **Flight Booking Tool** – Books a flight between two cities.
2.  **Weather API Tool** – Checks weather in Los Angeles.
3.  **Packing Suggestion Tool** – Suggests what to pack based on weather.

Each of these is a **Tool** – they do one specific job.


###  **Agent's Role**:

The **Agent** will:

1. Understand your request.
2. Use the **Flight Booking Tool** to book the flight.
3. Use the **Weather Tool** to check LA’s weather.
4. Use the **Packing Tool** to suggest clothes.
5. Combine all results and give you one complete answer.


###  **Agent in Action:**

Here’s how it thinks step-by-step:

>  “I need to book a flight” → Use **Flight Tool**
>  “I need the weather in LA” → Use **Weather Tool**
>  “I need to suggest what to pack” → Use **Packing Tool**
>  Then combine everything into one smart reply!


### **Final Answer from the Agent**:

> "Your flight from New York to Los Angeles is booked for May 30. The weather in LA is sunny with highs of 28°C. You should pack light clothes, sunglasses, and sunscreen."


### Summary:

| Role      | Example                                                                       |
| --------- | ----------------------------------------------------------------------------- |
| **Tool**  | Does one job (e.g., checks weather)                                           |
| **Agent** | Understands the full task, uses tools in the right order, and replies smartly |

